# Image Capturing


In [5]:
# get jetbot instance to start 
from jetbot import Robot

robot = Robot()

In [6]:
# Create and display Image widget
import ipywidgets.widgets as widgets

# First, let's display an ``live_camera`` widget that we'll use to show our live camera feed.
live_camera = widgets.Image(format='jpeg', width=300, height=300)

# second, let's display an ``snapsot_image`` widget that we'll use to show snapshot captured.
snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

In [7]:
# create a functions to handle robot according to signs

def stop():
    robot.stop()
        
def step_forward():
    robot.forward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_backward():
    robot.backward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_left():
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right():
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

# map the inputs to the function blocks
move_options = {1 : step_forward,
           2 : step_backward,
           3 : step_right,
           4 : step_left,
           5 : stop,
}

# function to send picture to machine learning model for object detection. which will return a codeaccording to sign
def check_sign(file_name , count):
    return 1


In [8]:
# get a camera instance to attach with 'live_camera' widget
from jetbot import Camera

# we call the ``instance`` method which will create a new camera if it hasn't been created yet. 
#If once already exists, this method will return the existing camera.
camera = Camera.instance() 

### Connect Camera to Image widget

Our camera class currently only produces values in BGR8 (blue, green, red, 8bit) format, while our image widget accepts values in compressed *JPEG*.
To connect the camera to the image we need to insert the ``bgr8_to_jpeg`` function as a transform in the link.  We do this below

In [10]:
from jetbot import bgr8_to_jpeg
import traitlets

camera_link = traitlets.dlink((camera, 'value'), (live_camera, 'value'), transform=bgr8_to_jpeg)

In [11]:
# display all widgets
display(widgets.HBox([live_camera, snapshot_image]))

In [ ]:
#capture and save snapshots
import time
import os
from pathlib import Path

interval_seconds = 5
directory = 'snapshots'
Path(directory).mkdir(parents=True, exist_ok=True)

count = 0

while True:
    file_name = os.path.join(directory, 'image_%d.jpeg' % count)
    with open(file_name, 'wb') as f:
        f.write(bgr8_to_jpeg(camera.value))  
    # to check a sign using machine learning model
    sign = check_sign(file_name, count)
    move_options[sign]()
    # display snapshot that was saved
    snapshot_image.value = live_camera.value
    count +=  1
    time.sleep(interval_seconds)

### Stop robot if network disconnects

You can drive your robot around by looking through the video feed. But what if your robot disconnects from Wifi?  Well, the motors would keep moving and it would keep trying to stream video and motor commands.  Let's make it so that we stop the robot and unlink the camera and motors when a disconnect occurs.

In [ ]:
from jetbot import Heartbeat


def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

If the robot disconnects from the internet you'll notice that it stops.  You can then re-connect the camera and motors by re-creating the links with the cell below

In [ ]:
# only call this if your robot links were unlinked, otherwise we'll have redundant links which will double
# the commands transfered
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)